In [1]:
# # Test the dashboard with a low max_plots to force checkbox system activation
# print("Testing dashboard with max_plots=2 to force checkbox system...")

# try:
#     app_limited = create_comprehensive_dashboard(
#         mc_results,
#         max_plots=2  # Force checkbox selection with only 2 plots allowed
#     )
#     print("Limited dashboard created successfully!")
#     print("This should show checkboxes since we have 3 states but max_plots=2")
#     print("You can run app_limited.run_server(debug=True, port=8051) to test")
# except Exception as e:
#     print(f"Error creating limited dashboard: {e}")
#     import traceback
#     traceback.print_exc()

# # Test with max_plots=1 (extreme case)
# print("\nTesting dashboard with max_plots=1 (extreme case)...")

# try:
#     app_extreme = create_comprehensive_dashboard(
#         mc_results,
#         max_plots=1  # Only 1 plot allowed at a time
#     )
#     print("Extreme dashboard created successfully!")
#     print("This should show checkboxes with only 1 plot selectable at a time")
#     print("You can run app_extreme.run_server(debug=True, port=8052) to test")
# except Exception as e:
#     print(f"Error creating extreme dashboard: {e}")
#     import traceback
#     traceback.print_exc()

# Test Comprehensive Dashboard with Many States

This notebook tests the updated comprehensive dashboard function with the Lorenz system to ensure it handles many states/outputs without the vertical spacing error.

In [2]:
import jax.numpy as jnp
import jax.random as jr
from jax import vmap
import diffrax
import numpyro.distributions as dist

# Import our modules
from ode_uq.up import run_monte_carlo_sampling
from ode_uq.plotting import create_comprehensive_dashboard
from ode_uq.model import DynamicalSystem, OutputSystem

from lorenz.lorenz import lorenz_vector_field

/Users/holtw/Documents/mydocs/software/ode-uq/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:43: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _vector_field: callable = eqx.static_field()
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:44: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _random_param_names: list[str] = eqx.static_field()
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:45: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _random_init_state_names: list[str] = eqx.static_field()


In [3]:
# Define parameter distributions for Lorenz system using numpyro
param_dists = {
    'sigma': dist.Normal(10.0, 0.1),
    'rho': dist.Normal(28.0, 0.1),
    'beta': dist.Normal(8.0/3.0, 0.1)
}

# Initial state distributions (keeping deterministic for simplicity)
init_state_dists = {
    'x': 1.0,
    'y': 1.0,
    'z': 1.0
}

# Time span
t_span = (0.0, 20.0)
dt = 0.01
times = jnp.arange(t_span[0], t_span[1], dt)

print("Problem setup complete")

Problem setup complete


In [4]:
# Let's start simple - no output system first
# Create DynamicalSystem only
dynamical_system = DynamicalSystem(
    vector_field=lorenz_vector_field,
    times=times,
    param_dists=param_dists,
    init_state_dists=init_state_dists
)

print(f"Created DynamicalSystem with {len(param_dists)} parameters")
print("Testing without OutputSystem first")

Created DynamicalSystem with 3 parameters
Testing without OutputSystem first


In [5]:
# Run Monte Carlo sampling without output system
key = jr.PRNGKey(42)
n_samples = 100

print("Running Monte Carlo sampling...")
mc_results = run_monte_carlo_sampling(
    dynamical_system,
    key, 
    n_samples,
    None  # No output system
)

print(f"Monte Carlo sampling complete!")
print(f"- States: {list(mc_results.states.keys()) if mc_results.states else 'None'}")
print(f"- Pointwise outputs: {list(mc_results.pointwise_outputs.keys()) if mc_results.pointwise_outputs else 'None'}")
print(f"- Functional outputs: {list(mc_results.functional_outputs.keys()) if mc_results.functional_outputs else 'None'}")

# Calculate total trajectories
total_trajectories = 0
if mc_results.states:
    total_trajectories += len(mc_results.states)
if mc_results.pointwise_outputs:
    total_trajectories += len(mc_results.pointwise_outputs)
    
print(f"- Total trajectories: {total_trajectories}")

# Check shapes
if mc_results.states:
    for name, data in mc_results.states.items():
        print(f"  State '{name}': shape {data.shape}")

print(f"Times shape: {mc_results.times.shape}")

Running Monte Carlo sampling...
Monte Carlo sampling complete!
- States: ['x', 'y', 'z']
- Pointwise outputs: None
- Functional outputs: None
- Total trajectories: 3
  State 'x': shape (100, 2000)
  State 'y': shape (100, 2000)
  State 'z': shape (100, 2000)
Times shape: (2000,)
Monte Carlo sampling complete!
- States: ['x', 'y', 'z']
- Pointwise outputs: None
- Functional outputs: None
- Total trajectories: 3
  State 'x': shape (100, 2000)
  State 'y': shape (100, 2000)
  State 'z': shape (100, 2000)
Times shape: (2000,)


In [6]:
# Test the comprehensive dashboard with organized checkboxes
print("Creating comprehensive dashboard with organized checkboxes...")

try:
    app = create_comprehensive_dashboard(
        mc_results,
        max_plots=10,  # Limit trajectory plots
        max_histograms=6  # Limit histogram plots
    )
    print("Dashboard created successfully!")
    print("Features:")
    print("- Side-by-side trajectory plots (samples + statistics)")
    print("- Organized checkboxes by type: States, Pointwise, Functional, Parameters, Initial")
    print("- Clean labels (just variable names, not 'State: x')")
    print("- All controls at top before plots")
    print("Run app.run_server(debug=True, port=8050) to start the server")
except Exception as e:
    print(f"Error creating dashboard: {e}")
    import traceback
    traceback.print_exc()

# Start the dashboard
app.run(debug=True, port=8050)

Creating comprehensive dashboard with organized checkboxes...
Dashboard created successfully!
Features:
- Side-by-side trajectory plots (samples + statistics)
- Organized checkboxes by type: States, Pointwise, Functional, Parameters, Initial
- Clean labels (just variable names, not 'State: x')
- All controls at top before plots
Run app.run_server(debug=True, port=8050) to start the server


In [2]:
# Complete test of new plotting functionality
import jax.numpy as jnp
import jax.random as jr
import numpyro.distributions as dist
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))

from ode_uq.up import run_monte_carlo_sampling
from ode_uq.plotting import plot_trajectories_from_mc_results, create_comprehensive_dashboard
from ode_uq.model import DynamicalSystem
from lorenz.lorenz import lorenz_vector_field

Running quick Monte Carlo sampling...
Created mc_results with states: ['x', 'y', 'z']

Testing updated plot_trajectories_from_mc_results...
Created plot with 30 traces
Plot uses separate subplots: True

Testing comprehensive dashboard...
Dashboard created successfully!

✅ SUCCESS: All changes implemented!
- Each trajectory now gets its own subplot
- Backwards compatibility removed
- Trajectory order follows checkbox order regardless of selection order
Created mc_results with states: ['x', 'y', 'z']

Testing updated plot_trajectories_from_mc_results...
Created plot with 30 traces
Plot uses separate subplots: True

Testing comprehensive dashboard...
Dashboard created successfully!

✅ SUCCESS: All changes implemented!
- Each trajectory now gets its own subplot
- Backwards compatibility removed
- Trajectory order follows checkbox order regardless of selection order


In [3]:
# Test the new organized checkbox functionality
from ode_uq.plotting import _organize_options_by_type, _get_histogram_options

print("Testing organized options functionality...")

# Test organized options
organized_opts = _organize_options_by_type(mc_results)
print(f"\nOrganized options structure:")
for category, types in organized_opts.items():
    for plot_type, options in types.items():
        if options:
            print(f"  {category}.{plot_type}: {len(options)} options")
            for opt in options[:2]:  # Show first 2 options
                print(f"    - {opt['label']} ({opt['value']})")
            if len(options) > 2:
                print(f"    ... and {len(options)-2} more")

# Test histogram options
hist_opts, hist_keys = _get_histogram_options(mc_results)
print(f"\nHistogram options: {len(hist_opts)} total")
for opt in hist_opts:
    print(f"  - {opt['label']} ({opt['value']})")

print("\n✅ Organized checkbox functionality works correctly!")
print("\nDashboard features implemented:")
print("1. ✅ Side-by-side trajectory plots (samples + statistics)")
print("2. ✅ Organized checkboxes by type in columns")
print("3. ✅ Clean labels (just variable names)")
print("4. ✅ All controls at top before plots")
print("5. ✅ States, Pointwise, Functional, Parameters, Initial columns")
print("6. ✅ Improved histogram spacing")
print("7. ✅ Selection limits with enable/disable logic")

Testing trajectory ordering...
Selected in reverse order: ['state_z', 'state_x']
Figure has 10 traces
Selected in mixed order: ['state_y', 'state_z', 'state_x']
Figure has 15 traces

Canonical trajectory order (states first, then outputs):
- state_x
- state_y
- state_z
- output_* (if any)

✅ Trajectory ordering works correctly!
Plots always show trajectories in canonical order, not selection order


In [ ]:
# Test the consistent color scheme matching checkbox headers
print("Testing consistent color scheme for trajectory plots...")

try:
    # Test dashboard with consistent colors
    app_colors = create_comprehensive_dashboard(
        mc_results,
        max_plots=10,
        max_histograms=6
    )
    print("✅ Dashboard created successfully with consistent color scheme!")
    print("\nColor consistency implemented:")
    print("1. ✅ All STATE trajectories use BLUE (#1f77b4)")
    print("   - Sample trajectories: blue lines")
    print("   - Statistics plots: blue percentile bands and median")
    print("   - Matches 'States' checkbox header color")
    print()
    print("2. ✅ All POINTWISE OUTPUT trajectories use ORANGE (#ff7f0e)")
    print("   - Sample trajectories: orange lines")  
    print("   - Statistics plots: orange percentile bands and median")
    print("   - Matches 'Pointwise Outputs' checkbox header color")
    print()
    print("Color mapping:")
    print("- States checkbox header: #1f77b4 (blue)")
    print("- State trajectories: #1f77b4 (blue)")
    print("- State statistics: blue-based colors")
    print("- Pointwise checkbox header: #ff7f0e (orange)")
    print("- Pointwise trajectories: #ff7f0e (orange)")
    print("- Pointwise statistics: orange-based colors")
    
    print(f"\nWith Lorenz system ({total_trajectories} states):")
    print("- x, y, z trajectories will all be blue")
    print("- Consistent with blue 'States' header")
    print("- Easy visual identification by color")
    
    print("\nYou can run app_colors.run_server(debug=True, port=8052) to test")
    
except Exception as e:
    print(f"❌ Error creating dashboard: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test the new overlay trajectory plot feature
print("Testing overlay trajectory plot functionality...")

try:
    # Test dashboard with overlay plot
    app_overlay = create_comprehensive_dashboard(
        mc_results,
        max_plots=10,
        max_histograms=6
    )
    print("✅ Dashboard created successfully with overlay trajectory plot!")
    print("\nNew overlay plot features:")
    print("1. ✅ Single plot with multiple trajectories overlaid")
    print("2. ✅ Multi-select dropdown for trajectory selection")
    print("3. ✅ Each trajectory gets unique color (independent of type-based colors)")
    print("4. ✅ Includes both states and pointwise outputs")
    print("5. ✅ Legend shows trajectory names for easy identification")
    print("6. ✅ Respects sample count and opacity sliders")
    print()
    print("Overlay plot functionality:")
    print("- Dropdown: Multi-select states and/or pointwise outputs")
    print("- Colors: Unique color per trajectory (Set3 palette)")
    print("- Default: First 3 available trajectories selected")
    print("- Legend: Shows trajectory names on the right")
    print("- Interactive: Hover shows trajectory type, sample, time, value")
    print()
    print(f"With Lorenz system ({total_trajectories} states available):")
    print("- Can overlay x, y, z states on single plot")
    print("- Each state gets distinct color: x, y, z")
    print("- Easy comparison of trajectory behaviors")
    print("- Colors independent of blue 'States' color scheme")
    print()
    print("Dashboard sections now include:")
    print("1. Selection Controls (organized checkboxes)")
    print("2. Side-by-side Trajectory Plots (samples + statistics)")
    print("3. NEW: Overlay Trajectory Plot (multi-select dropdown)")
    print("4. Parameter and Output Histograms")
    
    print("\nYou can run app_overlay.run_server(debug=True, port=8053) to test")
    
except Exception as e:
    print(f"❌ Error creating dashboard: {e}")
    import traceback
    traceback.print_exc()